<a href="https://colab.research.google.com/github/gt-cse-6040/bootcamp/blob/main/Module%201/Session%204/m1s4nb2_SQL_intro_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Main topics covered during today's session:_

Previous NB:

1. **Intro to SQL:**
    
    a. SQL clauses
    
    b. Joins
    
    c. How to write SQL queries and use sqlite3 in NB's
    
    d. DB Browser for SQLite
    
This NB:

2. **Querying in SQL**
    
    a. Aggregate function overview
    
    b. SQL query to pandas dataframe
    
    c. String manipulations, in select and sorting/joining


# Intro to SQL, Part 2

In [ ]:
!wget https://github.com/gt-cse-6040/bootcamp/raw/main/Module%201/Session%204/NYC-311-2M_small.db

## Aggregate Functions

#### Recall that aggregate functions perform a specific operations over all of the rows in a group (group by clause). Aggregate functions differ from other functions in that they take many rows of input and return a single row of output.

**Keep in mind that aggregate functions (typically) ignore NULL values.**

In [ ]:
%%html
<style>
table {float:left}
</style>

The following table summarizes some useful SQL aggregations:

| Aggregate Function       | Description                       |
|--------------------------|-----------------------------------|
| ``COUNT( * )``           | total number (count) of all rows  |
| ``COUNT( value )``       | counts all non-NULL rows          |
| ``AVG( value )``         | averages all non-NULL values      |
| ``MIN( value )``         | returns the lowest value          |
| ``MAX( value )``         | returns the highest value         |
| ``TOTAL( value )``       | returns sum of all non-NULL values|
| ``SUM (value )``         | returns sum of all non-NULL values|



**A few notes about SUM() and TOTAL():**

The sum() and total() aggregate functions return the sum of all non-NULL values in the group. If there are no non-NULL input rows then sum() returns NULL but total() returns 0.0.

The result of total() is always a floating point value.

The result of sum() is an integer value if all non-NULL inputs are integers. If any input to sum() is neither an integer nor a NULL, then sum() returns a floating point value which is an approximation of the mathematical sum.

https://www.sqlite.org/lang_aggfunc.html

#### SQLite does not include a whole lot of aggregate functions, as you can see.

What that means is that pandas will be your preferred method for more advanced aggregations, such as median, standard deviation, and other statistical functions.

Let's look at some examples from the NYC 311 Calls database in Notebook 9.

For this exercise we have a subset of data, consisting of one month from 2014.

In [ ]:
# create a connection to the database
import sqlite3 as db

# Connect to a database (or create one if it doesn't exist)
conn = db.connect('NYC-311-2M_small.db')

Opening this database in DB Browser, we can see that the columns ClosedDate and City have NULL values. Let's see how the COUNT() function differs.

In [ ]:
# count(*) returns all rows, including NULLS
query_nulls = '''
SELECT COUNT(*)

FROM DATA

'''

In [ ]:
# execute the query
c = conn.cursor()
c.execute(query_nulls)
results_nulls = c.fetchall()
results_nulls

In [ ]:
# count only the non-NULL rows
query_ClosedDate = '''
SELECT COUNT(ClosedDate)

FROM DATA

'''

In [ ]:
# execute the query
c = conn.cursor()
c.execute(query_ClosedDate)
results_ClosedDate = c.fetchall()
results_ClosedDate

We can see from DB Browser that there are 2 million rows in the table, and the COUNT( * ) included all of them, while there are NULL values in the ClosedDate column, so the COUNT of those values is somewhat less.

Now let's look at a simple GROUP BY (again from NB 9 Part 1).

In [ ]:
query_group = '''
SELECT ComplaintType, Descriptor, Agency, COUNT(*)

FROM data

GROUP BY ComplaintType

LIMIT 10
'''

In [ ]:
# execute the query
c = conn.cursor()
c.execute(query_group)
results_group = c.fetchall()
results_group

#### We can see a few complexities with executing the query in this manner.

1. The data comes back as a list of tuples. With small datasets, this might be ok, but with larger datasets this becomes a bit unwieldy.

2. The results do not include any column names.

3. Further manipulation of the results requires us to code for nested data.

**So bringing back your query results to a pandas dataframe is much better.**

**Pandas has a built in SQL query reader, which sends the query to the database and returns a dataframe of the results.**

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_sql_query (query_group, conn)
df

#### Now let's look at string manipulation functions in SQL.

https://www.sqlitetutorial.net/sqlite-string-functions/

Some of the ones that we will use in this class are UPPER, LOWER, and SUBSTR.

The string functions generally work in the same manner as their Python equivalents, just check the documentation for the specific syntax.

Let's look at the UPPER function for some specific things that you should know.

In [ ]:
query_upper = '''
SELECT DISTINCT ComplaintType, UPPER(ComplaintType) as UPPER_CASE

FROM data

LIMIT 10
'''

In [ ]:
df1 = pd.read_sql_query (query_upper, conn)
df1

Seems simple, yes?

But the string functions take on a bit more complexity when you are doing aggregations.

Exercises 1, 2, and 4 in Notebook 9 Part 1 provide some good examples of how this works.

**Exercise 1** (2 points). Create a string, `query`, containing an SQL query that will return the number of complaints by type. The columns should be named `type` and `freq`, and the results should be sorted in descending order by `freq`. Also, since we know some complaints use an inconsistent case, for your function convert complaints to lowercase.

In [ ]:
### BEGIN SOLUTION
query = '''
SELECT LOWER(ComplaintType) AS type, COUNT(*) as freq

FROM data

GROUP BY type

ORDER BY -freq
'''
### END SOLUTION

# Runs your `query`:
df_complaint_freq = pd.read_sql_query(query, conn)
df_complaint_freq.shape

Note that our grouping is done on the 'type' alias, and not on ComplaintType.

What would happen if we simply grouped on the ComplaintType?

In [ ]:
### what about this one?
query2 = '''
SELECT LOWER(ComplaintType) AS type, COUNT(*) as freq

FROM data

GROUP BY ComplaintType

ORDER BY -freq
'''
###

# Runs your `query`:
df_complaint_freq2 = pd.read_sql_query(query2, conn)
df_complaint_freq2.shape

Execute this query in DB Browser:

SELECT DISTINCT ComplaintType

FROM data

WHERE ComplaintType like '%elevator%' or ComplaintType like '%plumbing%'

ORDER BY ComplaintType

**Exercise 2** (2 points). Create a string variable, `query`, that contains an SQL query that will return the top 10 cities with the largest number of complaints, in descending order. It should return a table with two columns, one named `name` holding the name of the city, and one named `freq` holding the number of complaints by that city.

Like complaint types, cities are not capitalized consistently. Therefore, standardize the city names by converting them to **uppercase**.

In [ ]:
### BEGIN SOLUTION
query3 = '''
  SELECT UPPER(City) AS name, COUNT(*) AS freq
    FROM data
    GROUP BY name
    ORDER BY -freq
    LIMIT 10
'''
### END SOLUTION

# Runs your `query`:
df_whiny_cities = pd.read_sql_query(query3, conn)

In [ ]:
### what about this one?
query4 = '''
  SELECT UPPER(City) AS name, COUNT(*) AS freq
    FROM data
    GROUP BY City
    ORDER BY -freq
    LIMIT 10
'''
### END SOLUTION

# Runs your `query`:
df_whiny_cities1 = pd.read_sql_query(query4, conn)

In [ ]:
display(df_whiny_cities)
display(df_whiny_cities1)

Again, execute the below in DB Browser:

SELECT DISTINCT City

FROM data

WHERE City like '%astoria%' or City like '%flushing%'

ORDER BY City

**The next cell is here simply to put in place the requirements for exercise 4, which we are working through below.**

In [ ]:
query = '''
  SELECT UPPER(City) AS name, COUNT(*) AS freq
    FROM data
    WHERE name <> 'None'
    GROUP BY City COLLATE NOCASE
    ORDER BY -freq
    LIMIT 10
'''
df_whiny_cities2 = pd.read_sql_query(query, conn)
TOP_CITIES = list(df_whiny_cities2.head(7)['name'])
# display(TOP_CITIES)

# solution for exercise 3, to set up what is needed for exercise 4
def strs_to_args(str_list):
    assert type (str_list) is list
    assert all ([type (s) is str for s in str_list])
    ### BEGIN SOLUTION
    quoted = ['"{}"'.format(s) for s in str_list]
    return ', '.join(quoted)
    ### END SOLUTION
TOP_CITIES_as_args = strs_to_args(TOP_CITIES)
TOP_CITIES_as_args

#### The notebook shows another way to handle the case differences, using the `COLLATE NOCASE` keywords.

**Case-insensitive grouping: `COLLATE NOCASE`.** Another way to carry out the preceding query in a case-insensitive way is to add a `COLLATE NOCASE` qualifier to the `GROUP BY` clause.

The next example demonstrates this clause. Note that it also filters out the 'None' cases, where the `<>` operator denotes "not equal to." Lastly, this query ensures that the returned city names are uppercase.

> The `COLLATE NOCASE` clause modifies the column next to which it appears. So if you are grouping by more than one key and want to be case-insensitive, you need to write, `... GROUP BY ColumnA COLLATE NOCASE, ColumnB COLLATE NOCASE ...`.

**Exercise 4** (3 points). Suppose we want to look at the number of complaints by type _and_ by city **for only the top cities**, i.e., those in the list `TOP_CITIES` computed above. Execute an SQL query to produce a tibble named `df_complaints_by_city` with the variables {`complaint_type`, `city_name`, `complaint_count`}.

In your output `DataFrame`, convert all city names to uppercase and convert all complaint types to lowercase.

In [ ]:
### BEGIN SOLUTION
# Version 0:
query0 = """
SELECT LOWER(ComplaintType) AS complaint_type,
        UPPER(City) AS city_name,
        COUNT(*) AS complaint_count

FROM data

WHERE city_name IN ({})

GROUP BY City COLLATE NOCASE, ComplaintType COLLATE NOCASE

ORDER BY city_name, complaint_type, complaint_count

""".format(strs_to_args(TOP_CITIES))

# Version 1:
query1 = """
SELECT LOWER(ComplaintType) AS complaint_type,
        UPPER(City) AS city_name,
        COUNT(*) AS complaint_count

FROM data

WHERE city_name IN ({})

GROUP BY city_name, complaint_type

ORDER BY city_name, complaint_type, complaint_count

""".format(strs_to_args(TOP_CITIES))

df_complaints_by_city = pd.read_sql_query(query0, conn)
### END SOLUTION

# Previews the results of your query:
print("Found {} records.".format(len(df_complaints_by_city)))

See the two ways of handling the case-sensitive grouping. The first query uses `COLLATE NOCASE` and the second uses the column aliases.

So what happens if we don't handle the case-sensitivity?

In [ ]:
query2 = """
SELECT LOWER(ComplaintType) AS complaint_type,
        UPPER(City) AS city_name,
        COUNT(*) AS complaint_count

FROM data

WHERE city_name IN ({})

GROUP BY City, ComplaintType

ORDER BY city_name, complaint_type, complaint_count

""".format(strs_to_args(TOP_CITIES))

df_complaints_by_city2 = pd.read_sql_query(query2, conn)

# Previews the results of your query:
print("Found {} records.".format(len(df_complaints_by_city2)))

Without going into the details, we can see that the City and ComplaintType differences give different results.

The reason is that we did not account for case sensitivity in our groupings.

**So the takeaway is that we must ensure that we are correctly accounting for the data differences.**

Grouping in SQL is case-sensitive, so we must ensure that our code recognizes and deals with this.

**This is a simple example of 'dirty data', which is something that you will need to deal with throughout your Analytics career.**